# » `Dependencias`:

In [ ]:
# !pip install plotly
# !pip install sqlalchemy
# !pip install numpy
# !pip install matplotlib

In [1]:
import pandas as pd
from sqlalchemy import create_engine

## 1. Conección → SIM(172.27.0.124)

In [2]:
SERVER = '172.27.0.124' # '172.27.0.242'
DRIVER = 'ODBC Driver 17 for SQL Server'
DATABASE = 'SIM'
USERNAME = 'userestadistica' # 'udesa'
PASSWORD = '$Us3R_3sT4d1sTic4$' # 'DESARROLLO2006'
DATABASE_CONNECTION = f'mssql://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER}'

engine = create_engine(DATABASE_CONNECTION)
connection = engine.connect()

## 2. Métodos genéricos:

In [3]:
def get_query_sql(query):
  try:
    df = pd.read_sql(query, connection)
    return df
  except:
    print('¡Ocurrió un error!')

## 3. EXTRACCIÓN Y EXPLORACIÓN DE DATOS:

### 3.1 Exportar solicitudes CUE en etapa de `Evaluación` ...

In [4]:
from datetime import date, timedelta

year_today = date.today().year
last_month = 7 # date.today().month - 7

QUERY_SQL = f''' 

                  SELECT e.* FROM (
                  
                     SELECT
                        [Id Solicitud Cue] = s.nIdSolicitudCue,
                        [Login Usuario Analisis] = (-- nIdUsrInicia
                                             CASE WHEN S.nIdEtapaCUE = 3
                                                THEN (
                                                         SELECT TOP 1 CAST(ISNULL((SELECT sLogin FROM SimUsuario 
                                                                                 WHERE nIdOperador = CAST(E.nIdUsrFinaliza AS int)),'') AS VARCHAR(10)) 
                                                         FROM SimEtapaSolicitudCUE E 
                                                         WHERE 
                                                            E.nIdSolicitudCUE = S.nIdSolicitudCue 
                                                            AND E.nIdEtapaCUE = 2 
                                                            AND (E.sEstado ='F' OR E.sEstado ='S') 
                                                            AND e.bactivo = 1 
                                                         ORDER BY nidetapaSolicue DESC
                                                      )
                                                ELSE
                                                      CASE WHEN S.nIdEtapaCUE = 2 THEN -- ANALISIS
                                                         CASE 
                                                            WHEN s.sEstadoActualSoliCUE = 'F' THEN (SELECT TOP 1 CAST(ISNULL((SELECT sLogin FROM SimUsuario WHERE nIdOperador= CAST(E.nIdUsrFinaliza AS int)),'') AS VARCHAR(10)) FROM SimEtapaSolicitudCUE E WHERE E.nIdSolicitudCUE = S.nIdSolicitudCue AND E.nIdEtapaCUE = 2 and E.sEstado ='F' and e.bactivo = 1 ORDER BY nidetapaSolicue DESC) --queda
                                                            WHEN s.sEstadoActualSoliCUE = 'S' THEN (SELECT TOP 1 CAST(ISNULL((SELECT sLogin FROM SimUsuario WHERE nIdOperador= CAST(E.nIdUsrFinaliza AS int)),'') AS VARCHAR(10)) FROM SimEtapaSolicitudCUE E WHERE E.nIdSolicitudCUE = S.nIdSolicitudCue AND E.nIdEtapaCUE = 2 and E.sEstado ='S' and e.bactivo = 1 ORDER BY nidetapaSolicue DESC) --queda
                                                            WHEN s.sEstadoActualSoliCUE = 'I' THEN (SELECT sLogin FROM SimUsuario WHERE nIdOperador= CAST(S.nIdOperadorCue AS int)) 
                                                            WHEN s.sEstadoActualSoliCUE = 'O' THEN (SELECT sLogin FROM SimUsuario WHERE nIdOperador= CAST(S.nIdOperadorCue AS int)) 
                                                         END							
                                                      ELSE '' 
                                                      END 	
                                             END
                        ),
                        [Documento] = ISNULL(s.sIdDocumento, ''),
                        [Num Documento] = ISNULL(s.sNumDocumento, ''),
                        [Nombre] = s.sNombre,
                        [Primer Apellido] = s.sPrimerApellido,
                        [Segundo Apellido] = s.sSegundoApellido,
                        [Sexo] = p.sSexo,
                        [Fecha Nacimiento] = p.dFechaNacimiento,
                        [Estado Civil] = p.sIdEstadoCivil,
                        [Pais Nacimiento] = p.sIdPaisNacimiento,
                        [Pais Nacionalidad] = p.sIdPaisNacionalidad,
                        [Num Solicitud Cue] = s.sNumSolicitudCue,
                        [Fecha Solicitud] = CAST(s.dFechaSolicitud AS DATE),
                        [Tipo Tramite] = tt.sDescripcion,
                        [Número Trámite] = s.sNumeroTramite,
                        [¿Presentó DJ?] = (
                                             IIF(
                                                EXISTS(
                                                   SELECT TOP 1 1
                                                   FROM SimDjIdentidadesPersonaCUE dj
                                                   WHERE
                                                      dj.bActivo = 1
                                                      AND dj.sNumeroTramite = s.sNumeroTramite
                                                ),
                                                'Si',
                                                'No'
                                             )
                                       ),
                        [Etapa CUE] = (SELECT sDescripcion FROM SimEtapaCUE WHERE nIdEtapaCUE= S.nIdEtapaCUE),
                        [Cantidad Coincidencias(CUE)] = (
                                                   SELECT COUNT(1) FROM SimCoincidenciasIdentidadesCUE ci
                                                   WHERE 
                                                      ci.bActivo = 1
                                                      AND ci.nIdSolicitudCue = s.nIdSolicitudCue
                        ),
                        [Cantidad Evaluación] = (
                                                   SELECT COUNT(1)
                                                   FROM SimCoincidenciasIdentidadesCUE c
                                                   WHERE 
                                                      c.bActivo = 1
                                                      AND c.bEvaluacion = 1
                                                      AND c.nIdSolicitudCue = s.nIdSolicitudCue
                                       ),
                        [Cantidad Union Personas] = (
                                                      SELECT COUNT(1)
                                                      FROM SimCoincidenciasIdentidadesCUE c
                                                      WHERE 
                                                         c.bActivo = 1
                                                         AND c.bUnionIdPersona = 1
                                                         AND c.nIdSolicitudCue = s.nIdSolicitudCue
                                       ),
                        [Fecha Evaluación (F)] = (
                                                         SELECT ses2.dFechaHoraFin FROM (
                                                            SELECT
                                                               es.dFechaHoraFin,
                                                               [nOrden] = ROW_NUMBER() OVER (ORDER BY es.nIdEtapaSoliCUE DESC)
                                                            FROM SimEtapaSolicitudCUE es
                                                            WHERE
                                                               es.bActivo = 1
                                                               AND es.nIdSolicitudCUE = s.nIdSolicitudCue
                                                               AND es.nIdEtapaCUE = 3 -- 3 ↔ EVALUACIÓN
                                                         ) ses2
                                                         WHERE
                                                            ses2.nOrden = 1
                                                      ),
                        [¿Observado en Evaluación?] = (
                                                         IIF(
                                                            EXISTS(
                                                               SELECT TOP 1 1
                                                               FROM SimSolicitudObsCUE so
                                                               WHERE 
                                                                  so.bActivo = 1
                                                                  AND so.nIdEtapaCUE = 3 -- Evaluación
                                                                  AND so.nIdSolicitudCUE = s.nIdSolicitudCUE
                                                            ),
                                                            'Si',
                                                            'No'
                                                         )
                        ),
                        [Detalle Observacion] = (
                                                   REPLACE(
                                                      REPLACE(
                                                         (
                                                            SELECT so.sObservacion
                                                            FROM SimSolicitudObsCUE so
                                                            WHERE 
                                                               so.bActivo = 1
                                                               AND so.nIdEtapaCUE = 3 -- Evaluación
                                                               AND so.nIdSolicitudCUE = s.nIdSolicitudCUE
                                                            FOR XML PATH('')
                                                         ),
                                                         '<sObservacion>',
                                                         'Observación: '
                                                      ),
                                                      '</sObservacion>',
                                                      '; '
                                                   )
                        ),
                        [Estado Actual Soli CUE] = (
                              CASE
                                 WHEN s.sEstadoActualSoliCUE = 'I' THEN 'INICIADO'
                                 WHEN s.sEstadoActualSoliCUE = 'O' THEN 'OBSERVADO'
                                 WHEN s.sEstadoActualSoliCUE = 'S' THEN 'SUBSANADO'
                                 WHEN s.sEstadoActualSoliCUE = 'F' THEN 'FINALIZADO'
                              END
                           )
                        
                     FROM SimSolicitudCUE s
                     JOIN SimTipoTramite tt ON tt.nIdTipoTramite = s.nIdTipoTramite
                     JOIN SimPersona p ON s.uIdPersona = p.uIdPersona
                     WHERE 
                        s.bActivo = 1
                        AND s.nIdEtapaCUE = 3 -- EVALUACIÓN
                        AND s.sEstadoActualSoliCUE = 'F'
                        AND (s.nProcesoCoincidencias = 1 AND s.nProcesoHuellas = 1)

                  ) e
                  WHERE 
                     YEAR(e.[Fecha Evaluación (F)]) = { year_today } 
                     AND MONTH(e.[Fecha Evaluación (F)]) = { last_month }

 '''

df_cue_evaluacion = get_query_sql(QUERY_SQL)


In [5]:
from datetime import date

root_path = r'D:\srim_reportes\cue\evaluados\gcampos'
file_name = f'{year_today}-{str(last_month).zfill(2)}'
full_path = f'{root_path}\\{file_name}.xlsx'

df_cue_evaluacion.to_excel(full_path, sheet_name=file_name)